#SocialAL Model
#generate trial-by-trial output from model fits

KLS 2.7.20
Project info: https://osf.io/b48n2/

Model modified from : Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

Python version and modules

In [1]:
import sys
print(sys.version)  
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import os
from decimal import *

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


Define Functions

In [26]:
def update_value(Prob):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
    EV = [(retain[x] + Prob*shared[x]) for x in range(0,4)]
    return EV

def update_prob(recip, Prob, a_gain, a_loss):
    gain = max(recip - Prob, 0)
    loss = min(recip - Prob, 0)
    Prob = Prob + a_gain * gain + a_loss * loss
    return Prob

def get_action_selection_prob(beta, EV, choice):
    actionProb = Decimal(np.exp(beta*EV[choice-1])/np.sum([np.exp(beta*x) for x in EV]))
    return actionProb

def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

def get_likelihood_action(params, data):
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    
    # initialize variables
    prob = [0.5, 0.5, 0.5]
    ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    tletters = ["A", "B", "C"]
    
    totalLLH = 0 
    for trial in range(0, len(data)):
        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        if choice != 0:
            # use the probability of the selection (choice-probability) to update log likelihood
            cprob = probs[choice-1] #print(cprob, isinstance(cprob, float))
            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
            

            # update prob and value
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], a_gain, a_loss)
            ev[trustee] = update_value(prob[trustee])
            
        #print("Trial #: ", trial + 1)
        #print("Trustee: ", tletters[trustee])
        print('LLH: ', totalLLH)
        #print('Trustee Prob: ', prob[trustee])
        #print('Trustee Expected values: ', ev[trustee])
        
    return totalLLH

def model_fit_once(data, param):
        # initialize free parameters with randomly chosen numbers
        a_gain=param[0]
        a_loss=param[1]
        beta=param[2]
        params = [a_gain, a_loss, beta]
        
        #results = minimize(get_likelihood_action, 
                           #params, args =(data), method='BFGS', options = {'maxiter': 10000, 'disp': False})
        results = minimize(get_likelihood_action, 
                       params, args =(data), bounds = [(0, 1), (0, 1), (1e-10, 100)], 
                       options = {'maxiter': 10000, 'disp': False})
        return results

Get data

In [23]:
#dt = pd.read_csv('../data/modeling/sub-1004.csv')
#dt = pd.read_csv('../data/modeling/sub-1008.csv') #- try getting this sub to work
dt = pd.read_csv('../data/modeling/sub-2032.csv') #- try getting this sub to work
# recode trial type into numbers for model
def stims(trial_type):
    if trial_type == "Trustworthy":
        return 0
    elif trial_type == "Neutral":
        return 1
    elif trial_type == "Untrustworthy":
        return 2
dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
# rename response_key to choice
def choices(response_key):
    if response_key == 'None':
        return 0 
    else:
        return response_key  
dt['Choice'] = dt['response_key'].apply(choices)
dt['Choice'] = pd.to_numeric(dt['Choice'])
# calculte the trustee response
def resp(trial_earnings):
    if trial_earnings >= 12:
        return 1
    else:
        return 0
dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
#print(data)

Input parameters

In [27]:
array = [ 0.49695529,  0.        ,  0.10735091, 54.29839312]


In [28]:
model_fit_once(data, array)

LLH:  0
LLH:  1.3862943611198906
LLH:  1.3862943611198906
LLH:  2.772588722239781
LLH:  4.1588830833596715
LLH:  5.545177444479562
LLH:  7.027446155658433
LLH:  7.027446155658433
LLH:  8.509714866837305
LLH:  9.896009227957196
LLH:  11.438456393314572
LLH:  12.740347553851722
LLH:  14.126641914971612
LLH:  15.428533075508762
LLH:  16.81482743662865
LLH:  17.876162592345572
LLH:  17.876162592345572
LLH:  18.889701364441503
LLH:  20.74640883708541
LLH:  22.132703198205302
LLH:  23.122925994200234
LLH:  24.509220355320124
LLH:  26.08487159444804
LLH:  27.06354754368398
LLH:  27.06354754368398
LLH:  28.053770339678913
LLH:  29.344124692302906
LLH:  30.730419053422796
LLH:  31.7033322400074
LLH:  33.089626601127286
LLH:  34.47592096224717
LLH:  35.44883414883177
LLH:  36.83512850995166
LLH:  38.221422871071546
LLH:  39.82800401741224
LLH:  40.79802983562016
LLH:  42.18432419674005
LLH:  43.47377748030631
LLH:  44.8600718414262
LLH:  46.14938020921787
LLH:  47.438688577009536
LLH:  49.370701

      fun: 54.29839311819601
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-7.10542736e-07,  2.53339039e+00,  1.42108547e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 12
      nit: 1
   status: 0
  success: True
        x: array([0.4969553, 0.       , 0.107351 ])